## 20 뉴스그룹 분류
: 18846개의 뉴스 문서를 20개의 뉴스 그룹 카테고리로 분류<br>
아래 표가 20개의 그룹들. (ex comp.graphics면 컴퓨터의 그래픽분야를 의미)<br> 
<img src=https://blog.kakaocdn.net/dn/lpcs3/btrFKmuULAV/9nVK96yU7tio7lv4WWwrB0/img.png width=1000>

**데이터 로딩과 데이터 구성 확인**

### 데이터 로드 및 확인

In [4]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
news_data = fetch_20newsgroups(subset='all',random_state=156)
# subset='train'이 디폴트이고 test 셋 가져오려면 'test'쓰면 된다.
## subset='all'쓰면 train과 test 전체 셋을 가져온다.

In [5]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [7]:
print(f'target 클래스의 값과 분포도 \n{pd.Series(news_data.target).value_counts().sort_index()}')
# 타겟은 0부터 19까지 20개의 값이고 각 타겟별, 값의 분포 확인 - 어느정도 고르게있다.

target 클래스의 값과 분포도 
0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64


In [4]:
print('target 0 ~ 19 클래스의 이름들  \n',news_data.target_names)
print(f"\n타겟 개수: {len(news_data.target_names)}, 타겟의 전체 행 개수: {pd.Series(news_data.target).shape}")

target 0 ~ 19 클래스의 이름들  
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']

타겟 개수: 20, 타겟의 전체 행 개수: (18846,)


In [5]:
# 첫번쨰 뉴스 문서 확인
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

### **학습과 테스트용 데이터 생성**

In [9]:
from sklearn.datasets import fetch_20newsgroups

# subset='train'으로 학습용(Train) 데이터만 추출, remove=('headers', 'footers', 'quotes')로 내용만 추출
## headers, footers, quotes만 보면 어떤 주제인지 너무 쉽게 파악할 수 있기 때문에 remove 파라미터를 통해 제외
## -> 본문 내용만 가지고 분류해볼 수 있게 진행
train_news= fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), random_state=156)
X_train = train_news.data
y_train = train_news.target
print(type(X_train))

# subset='test'으로 테스트(Test) 데이터만 추출, remove=('headers', 'footers', 'quotes')로 내용만 추출
test_news= fetch_20newsgroups(subset='test',remove=('headers', 'footers','quotes'),random_state=156)
X_test = test_news.data
y_test = test_news.target
print('학습 데이터 크기 {0} , 테스트 데이터 크기 {1}'.format(len(X_train) , len(X_test)))

<class 'list'>
학습 데이터 크기 11314 , 테스트 데이터 크기 7532


In [12]:
type(y_train)

numpy.ndarray

### **피처 벡터화 및 모델 학습/예측/평가**

**주의: 학습 데이터에 대해 fit( )된 CountVectorizer를 이용해서 테스트 데이터를 피처 벡터화 해야함.**<br>
<br>
**테스트 데이터에서 다시 CountVectorizer의 fit_transform()을 수행하거나 fit()을 수행 하면 안됨.**<br>
이는 이렇게 테스트 데이터에서 fit()을 수행하게 되면 기존 학습된 모델에서 가지는 feature의 갯수가 달라지기 때문임.

#### CountVectorizer

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization으로 feature extraction 변환 수행. 
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

# 학습 데이터로 fit( )된 CountVectorizer를 이용하여 테스트 데이터를 feature extraction 변환 수행. 
X_test_cnt_vect = cnt_vect.transform(X_test)

print(f'학습 데이터 Text의 CountVectorizer Shape: {X_train_cnt_vect.shape}') # -> sparse matrix 형태
print(f"테스트 데이터 Text의 CountVectorizer Shape: {X_test_cnt_vect.shape}")
# 테스트 데이터의 컬럼 개수는 101631로 학습데이터 컬럼수와 같다.
## 이유는 학습 데이터로 fit한 cnt_vect로 테스트 데이터에 대해 transform했기 때문이다.

학습 데이터 Text의 CountVectorizer Shape: (11314, 101631)
테스트 데이터 Text의 CountVectorizer Shape: (7532, 101631)


In [18]:
X_train_cnt_vect.toarray()[0]

array([0, 0, 0, ..., 0, 0, 0])

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

# LogisticRegression을 이용하여 학습/예측/평가 수행. 
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_cnt_vect , y_train)
pred = lr_clf.predict(X_test_cnt_vect)
print('CountVectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

CountVectorized Logistic Regression 의 예측 정확도는 0.617


#### TF-IDF
**TF-IDF 피처 변환과 머신러닝 학습/예측/평가**
- CounterVectorizer와 차이점은 단어가 한 문서에서 많이나오면 점수를 높게주되, 여러 문서에서 다같이 많이 나오면 패널티를 준다.

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF Vectorization 적용하여 학습 데이터셋과 테스트 데이터 셋 변환. 
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

# LogisticRegression을 이용하여 학습/예측/평가 수행. 
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_tfidf_vect , y_train) # 11314개의 문서(sparse_matrix)를 20개 토픽(0~19)로 fit
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

TF-IDF Logistic Regression 의 예측 정확도는 0.678


In [25]:
X_train_cnt_vect, y_train

(<11314x101631 sparse matrix of type '<class 'numpy.int64'>'
 	with 1103627 stored elements in Compressed Sparse Row format>,
 array([ 4, 15, 10, ...,  3,  9,  9]))

#### Stopwords 및 N-gram
**stop words 필터링을 추가하고 ngram을 기본(1,1)에서 (1,2)로 변경하여 피처 벡터화**

In [26]:
# stop words 필터링을 추가하고 ngram을 기본(1,1)에서 (1,2)로 변경하여 Feature Vectorization 적용.
# stop_words='english' - 영어 스탑워드 제외
# ngram_range=(1,2) - 1-gram과 2-gram 포함
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300 ) # max_df=300 - 300개 넘는 빈도수의 단어들은 제외
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_tfidf_vect , y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

TF-IDF Vectorized Logistic Regression 의 예측 정확도는 0.690


In [27]:
# ngram_range를 (1,2)로 해서 Tfidf 피처 벡터화 결과 생성된 피처 개수는
print(f"ngram_range를 (1,2)로 해서 Tfidf 피처 벡터화 결과 생성된 피처 개수: {X_train_tfidf_vect.shape[1]}")
tfidf_vect2 = TfidfVectorizer(stop_words='english', ngram_range=(1,1), max_df=300 ) # max_df=300 - 300개 넘는 빈도수의 단어들은 제외
tfidf_vect2_shape = tfidf_vect2.fit_transform(X_train).shape
print(f"ngram을 (1,1)로 줄이면 피처 개수는 확 줄어든다 -> {tfidf_vect2_shape[1]}")

ngram_range를 (1,2)로 해서 Tfidf 피처 벡터화 결과 생성된 피처 개수: 943453
ngram을 (1,1)로 줄이면 피처 개수는 확 줄어든다 -> 101039


#### GridSearchCV 하이퍼파라미터 튜닝
**GridSearchCV로 LogisticRegression C 하이퍼 파라미터 튜닝**<br>
**C: 1/alpha. (alpha :경사 하강법의 learning rate)**<br>
alpha와 규제의 강도는 비례하다. -> C와 규제의 강도는 반비례<br>
즉, C가 작아지면, 알파가 커지고, 규제를 쎄게한다는 뜻이다. C가 커지면, 알파가 작아지고, 규제를 약하게한다는 뜻이다.

In [14]:
from sklearn.model_selection import GridSearchCV

# 최적 C 값 도출 튜닝 수행. CV는 3 Fold셋으로 설정. 
params = { 'C':[0.01, 0.1, 1, 5, 10]} # 규제를 점차 약하게
grid_cv_lr = GridSearchCV(lr_clf ,param_grid=params , cv=3 , scoring='accuracy' , verbose=1 )
grid_cv_lr.fit(X_train_tfidf_vect , y_train)
print('Logistic Regression best C parameter :',grid_cv_lr.best_params_ )

# 최적 C 값으로 학습된 grid_cv로 예측 수행하고 정확도 평가. 
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
Logistic Regression best C parameter : {'C': 10}
TF-IDF Vectorized Logistic Regression 의 예측 정확도는 0.704


### 사이킷런 파이프라인(Pipeline) 사용 및 GridSearchCV와의 결합

#### 파이프라인

In [15]:
from sklearn.pipeline import Pipeline

# TfidfVectorizer 객체를 tfidf_vect 객체명으로, LogisticRegression객체를 lr_clf 객체명으로 생성하는 Pipeline생성
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)),
    ('lr_clf', LogisticRegression(solver='liblinear', C=10))
])

# 별도의 TfidfVectorizer객체의 fit_transform( )과 LogisticRegression의 fit(), predict( )가 필요 없음.
# pipeline의 fit( ) 과 predict( ) 만으로 한꺼번에 Feature Vectorization과 ML 학습/예측이 가능. 
## 즉, tfidf_vect로 fit_transform()하고 lr_clf로 fit 진행한다
pipeline.fit(X_train, y_train)

## 위에서 fit된 tfidf_vect로 transform하고, 학습된 lr_clf로 predict 진행한다.
pred = pipeline.predict(X_test) 
print('Pipeline을 통한 Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

Pipeline을 통한 Logistic Regression 의 예측 정확도는 0.704


#### 파이프라인과 GridSearchCV 결합
pipeline에 들어가는 params가 특이해짐 -> 언더바(_)2개로 하이퍼파라미터 설정

In [16]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english')),
    ('lr_clf', LogisticRegression(solver='liblinear'))
])

# Pipeline에 기술된 각각의 객체 변수에 언더바(_)2개를 연달아 붙여 GridSearchCV에 사용될 
# 파라미터/하이퍼 파라미터 이름과 값을 설정. . 
params = { 'tfidf_vect__ngram_range': [(1,1), (1,2)], # 전처리 하이퍼파라미터. ngram 범위
           'tfidf_vect__max_df': [100, 300, 700], # 전처리 하이퍼파라미터. 단어 최대 빈도수
           'lr_clf__C': [1, 5, 10] # 모델 학습 하이퍼파라미터. C(1/alpha) - 규제강도와 반비례
}

# GridSearchCV의 생성자에 Estimator가 아닌 Pipeline 객체 입력
grid_cv_pipe = GridSearchCV(pipeline, param_grid=params,cv=3 , scoring='accuracy', verbose=1)
grid_cv_pipe.fit(X_train, y_train)
print(grid_cv_pipe.best_params_, grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(X_test)
print('Pipeline을 통한 Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

Fitting 3 folds for each of 18 candidates, totalling 54 fits
{'lr_clf__C': 10, 'tfidf_vect__max_df': 700, 'tfidf_vect__ngram_range': (1, 2)} 0.7550828826229531
Pipeline을 통한 Logistic Regression 의 예측 정확도는 0.702
